## ACCRE


### Questions from Eric:

### 1) What groups are best optimizing their memory usage in terms of percent of actual memory used vs. memory requested for the job? What is the average percent for each group?

Note: 
    When Eric says "group" is he referring to "research groups" (which could be indicated by one or more account IDs), job groups (job IDs in an array), or partition groups?
    

### 2) Optimizing memory is more important for longer running jobs than shorter running jobs, as the resources are tied up for longer. If jobs are weighted by runtime, what is the average percent of memory used of the requested memory for each group? 
Note: Recall that high throughput computing (HTC) is focused on the use of multiple resources over long periods of time.

### 3) We are concerned with potentially unreliable notes which are not being detected by routine monitoring.  Of the jobs that failed, do any nodes show up unusually often?

Tip: Ignore debug partition. Look for failed jobs.

Note: Consider his use of "unusually often"

### 4) The CMS collaboration submits jobs that will run internal diagnostics and intentionally end early in 30 minutes. How often is this happening? (ie: what percent of jobs are CMS jobs) And does it happen on the same nodes repeatedly?

Tip: Check both 'production' and 'nogpfs' partitions. Look for commonly failing nodes and compare with other failed jobs. 

Note: 
    Filter df with below parameters:
        - Only production and nogpfs partitions 
        - State = 'cancelled'
        - Usedtime = 30 minutes
    Count number of unique job ids fall into this criteria
    Compare to total number of unique job ids

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
import re

#### Read in the data. The Nodes column triggers errors because several values are spread out over multiple other columns to the right.

In [2]:
#accre = pd.read_csv('../data/accre-jobs-2020.csv',names=['JOBID', 'ACCOUNT', 'USER', 'REQMEM', 'USEDMEM', 'REQTIME', 'USEDTIME',
       #'NODES', 'CPUS', 'PARTITION', 'EXITCODE', 'STATE', 'NODELIST'])

#MP's previous suggestion for how to read in the file by identifying which columns to include. Resulted in adding additional row with column names as row values

In [3]:
accre = pd.read_csv('../data/accre-jobs-2020.csv', nrows=9000, error_bad_lines=False)

b'Skipping line 3461: expected 13 fields, saw 15\nSkipping line 3462: expected 13 fields, saw 16\nSkipping line 3465: expected 13 fields, saw 15\nSkipping line 3468: expected 13 fields, saw 16\nSkipping line 3472: expected 13 fields, saw 15\nSkipping line 3476: expected 13 fields, saw 16\nSkipping line 3479: expected 13 fields, saw 14\nSkipping line 3482: expected 13 fields, saw 14\nSkipping line 3486: expected 13 fields, saw 14\nSkipping line 3490: expected 13 fields, saw 14\nSkipping line 3494: expected 13 fields, saw 14\nSkipping line 3495: expected 13 fields, saw 16\nSkipping line 3497: expected 13 fields, saw 14\nSkipping line 3498: expected 13 fields, saw 14\nSkipping line 3499: expected 13 fields, saw 14\nSkipping line 3505: expected 13 fields, saw 14\nSkipping line 3517: expected 13 fields, saw 16\nSkipping line 3526: expected 13 fields, saw 15\nSkipping line 3528: expected 13 fields, saw 15\nSkipping line 3529: expected 13 fields, saw 15\nSkipping line 3530: expected 13 fields

In [4]:
accre.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [5]:
#examining range of rows during group discussion
#accre.iloc[3449:3471,:]

In [6]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   JOBID      9000 non-null   object
 1   ACCOUNT    9000 non-null   object
 2   USER       9000 non-null   object
 3   REQMEM     9000 non-null   object
 4   USEDMEM    9000 non-null   object
 5   REQTIME    9000 non-null   object
 6   USEDTIME   9000 non-null   object
 7   NODES      9000 non-null   int64 
 8   CPUS       9000 non-null   int64 
 9   PARTITION  9000 non-null   object
 10  EXITCODE   9000 non-null   object
 11  STATE      9000 non-null   object
 12  NODELIST   9000 non-null   object
dtypes: int64(2), object(11)
memory usage: 914.2+ KB


### Convert column names to lowercase

In [8]:
#make column names lowercase, per group discussion
accre.columns = accre.columns.str.lower()

### Remove rows with values other than "production" in the Partition column.
"We really only want to look at the "production" partition so rows with other partitions should be removed"

#### However, we probably want to do this AFTER cleaning all the data. One of Eric's questions requests examining data where partition = 'nogpfs'

In [9]:
accre.partition.value_counts()

production        7274
turing            1253
nogpfs             435
sam                 12
debug               10
pascal               8
cgw-capra1           2
maxwell              2
cgw-cqs1             2
cgw-dougherty1       1
cgw-tbi01            1
Name: partition, dtype: int64

In [10]:
#use Boolean mask and save df back to itself.
accre = accre[accre['partition']=='production']

### Job time is in a format of either d-hh:mm:ss or hh:mm:ss. Convert to seconds.

In [11]:
#Convert the times into seconds

def convert_time_to_seconds(time):
    """Return an integer value of the time converted into seconds."""
    
    #Identify values not formatted as d-hh:mm:ss. Fill in the hour value by appending '0-' on the left of the string. 
    #This will make all our strings cleanly parse into an equal number of substrings.
    
    if len(time) <9:
        time = "0-" + time
    else: time
    
    #Use regex to select either '-' OR ':' as delimiter for split. The resulting substring list is ordered as [days, hours, minutes, seconds]
    #Cast the list of substrings to numeric, turning it into an array. Cleaner to do it here than for each individual substring
    time = pd.to_numeric(re.split('[-|:]',time))

    #Add the product of each unit and its multiplier to get the total time in seconds
    
    time_in_seconds = time[-4]*86400 + time[-3]*3600 + time[-2]*60 + time[-1]
    
    return time_in_seconds

In [12]:
#Change existing time columns to time in seconds.
accre['reqtime'] = accre['reqtime'].apply(convert_time_to_seconds)
accre['usedtime'] = accre['usedtime'].apply(convert_time_to_seconds)

In [13]:
#Create new column to reflect difference between requested time and used time.
accre['time_diff'] = accre['reqtime'] - accre['usedtime']

In [16]:
# Adjust order of columns in dataframe
accre = accre[['jobid', 'account', 'user', 'reqmem', 'usedmem', 'reqtime', 'usedtime','time_diff',
       'nodes', 'cpus', 'partition', 'exitcode', 'state', 'nodelist']]

In [17]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7274 entries, 0 to 8999
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   jobid      7274 non-null   object
 1   account    7274 non-null   object
 2   user       7274 non-null   object
 3   reqmem     7274 non-null   object
 4   usedmem    7274 non-null   object
 5   reqtime    7274 non-null   int64 
 6   usedtime   7274 non-null   int64 
 7   time_diff  7274 non-null   int64 
 8   nodes      7274 non-null   int64 
 9   cpus       7274 non-null   int64 
 10  partition  7274 non-null   object
 11  exitcode   7274 non-null   object
 12  state      7274 non-null   object
 13  nodelist   7274 non-null   object
dtypes: int64(5), object(9)
memory usage: 852.4+ KB


### Memory is reported in terms of Megabytes per node (Mc) or Megabytes per core (Mc), this needs to be uniformly converted to Megabytes per core by dividing by the number of cores per node in a job. 

Note: CPUS column includes total number of CPU cores allocated to a job (even for multi-node jobs). 
    WHEN reqmem column includes Mn AND in the entire usemem column
    Memory (CPUs / Nodes)


In [20]:
accre.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,time_diff,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,1188000,1188028,-28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,1188000,1090256,97744,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,1188000,1188020,-20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,1188000,1050632,137368,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,1188000,1188003,-3,1,24,production,0:0,COMPLETED,cn1440


### Slice off to create dataframe with records only related to node failures.
MP shared code on 10/3 

In [19]:
#accre[['exitcode_user', 'exitcode_node']] = accre['exitcode'].str.split(':', expand=True).astype(int)
#new DF only including records with node failures. 
#accre_node_failure = accre.loc[accre['exitcode_node'] != 0]

In [21]:
accre.NODES.nunique()

94

In [22]:
accre.NODES.value_counts()

1     3745514
1       65391
2        2459
3         861
4         579
       ...   
51          1
44          1
43          1
41          1
61          1
Name: NODES, Length: 94, dtype: int64

In [34]:
accre[accre.NODES==1].nunique()

JOBID        3745514
ACCOUNT          172
USER             594
REQMEM           204
USEDMEM       827081
REQTIME          448
USEDTIME      161540
NODES              1
CPUS              23
PARTITION         15
EXITCODE          24
STATE             33
NODELIST         868
dtype: int64

In [36]:
accre[(accre.STATE=='FAILED')|(accre.STATE=='OUT_OF_MEMORY')].shape

(1500, 13)

## Scraps 
To be deleted/saved elsewhere

In [108]:


#pd.to_datetime returned error Unknown string format 13-18:00:00.  Replace the - with :

#accre['reqtime'] = accre['reqtime'].str.replace('-',':')
#accre['usedtime'] = accre['usedtime'].str.replace('-',':')

#create a placeholder column in order to view the original and updated values side by side
#accre['reqtime2'] = accre['reqtime']

#accre['reqtime2'] = pd.to_timedelta(accre['reqtime2'])

#Write & test if/else portion of formula. IF the string value in the column is shorter than 9 characters, then it means there is no hour data.
#Fill in the hour value by appending '0-' on the left of the string. This will make all our strings able to parse into an equal number of substrings.

#time = '1-22:16:15'

#if len(time) <9:
    #time = "0-" + time
#else: time

#Use regex to select either '-' OR ':' as delimiter for split
#Cast substring to numeric, turning it into an array. Cleaner to do it here than for each individual substring

#substring = pd.to_numeric(re.split('[-|:]',time))
#substring

#Separate each element of the array and save to variable representing unit of time
#Add the product of each variable and the multiplier to get the total time in seconds

#day = substring[-4]
#hours = substring[-3]
#minutes = substring[-2]
#seconds = substring[-1]

#time_in_seconds= day*86400 + hours*3600 + minutes*60 + seconds


#sample_df = {'job':['1','2','3','4','5'],'reqtime_test':['23:58:00','2-10:58:00','2-00:00:00','22:18:36','4-02:50:00'],'usedtime_test':['22:19:15','1-23:58:00','1-05:17:04','20:52:47','4-02:50:01']}
#sample_df = pd.DataFrame(sample_df,columns=['job','reqtime_test','usedtime_test'])
#sample_df

#sample_df['reqtime_seconds'] = sample_df['reqtime_test'].apply(convert_time_to_seconds)

#sample_df

#Define time_in_seconds()

#def convert_time_to_seconds(time):
    #"""Return an integer value of the time converted into seconds."""
    
    #Identify values not formatted as dhhmmss. Fill in the hour value by appending '0-' on the left of the string. 
    #This will make all our strings able to parse into an equal number of substrings.
    
    #if len(time) <9:
        #time = "0-" + time
   # else: time
    
    #Use regex to select either '-' OR ':' as delimiter for split
    #Cast the list of substrings to numeric, turning it into an array. Cleaner to do it here than for each individual substring
    #time = pd.to_numeric(re.split('[-|:]',time))

    #Separate each element of the array and save to variable representing unit of time
    #day = time[-4]
    #hours = time[-3]
    #minutes = time[-2]
    #seconds = time[-1]

    #Add the product of each unit and it's multiplier to get the total time in seconds
    #time_in_seconds= day*86400 + hours*3600 + minutes*60 + seconds
    
    #return time_in_seconds